In [ ]:
import pandas as pd
import os

In [ ]:
df = pd.read_csv("labels/Anno_coarse/list_category_img.txt", delim_whitespace=True, header = 1)
df.head()

In [ ]:
for i in range(50):
    if not df[df.eq(i)['category_label']].empty :
        print(df[df.eq(i).any(axis=1)].head(1))
    else :
        print("No category " + str(i))

In [ ]:
category_df = pd.read_csv("labels/Anno_coarse/list_category_cloth.txt", delim_whitespace=True, header = 1)
category_df

In [ ]:
i = 0
for row in category_df.itertuples():
    if not df[df['image_name'].str.contains(str(row.category_name), regex=True)].empty:
        #print("Found " + row.category_name)
        category_df.at[row.Index, 'category_type'] = i
        #print(category_df.at[row.Index, 'category_type'])  # Affiche la valeur mise à jour
        i += 1
    else:
        print("Dropped " + row.category_name)
        category_df.drop(row.Index, inplace=True)
        
for row in category_df.itertuples():
    print(row)

In [ ]:
category_df

In [ ]:
def csv_to_txt(csv_path, txt_path):
    with open(csv_path, 'r') as csv_file, open(txt_path, 'w') as txt_file:
        for line in csv_file:
            txt_file.write(line)

In [ ]:
csv_filename = 'labels/Anno_coarse/categories.csv'
category_df.to_csv(csv_filename, index=False)
txt_filename = 'labels/Anno_coarse/categories.txt'
csv_to_txt(csv_filename, txt_filename)
os.remove(csv_filename)

In [ ]:
def list_directories(path):
    """ List all directories inside the given path using os module """
    return [name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))]

def directories_to_df(path):
    """ Create a DataFrame from the list of directory paths """
    directories = list_directories(path)
    df = pd.DataFrame(directories, columns=['image_name'])
    df['image_name'] = df['image_name'].astype(str)
    return df

In [ ]:
df_path = directories_to_df('data/img')
df_path

In [ ]:
adjectives = pd.DataFrame(df_path['image_name'].str.split('_', expand=True))
adjectives

In [ ]:
texture = adjectives[0].rename('texture')
texture = pd.DataFrame(texture.unique(), columns=['texture'])
texture.drop(axis=0, index=[0,1,2,3], inplace=True)
texture

In [ ]:
count_values = []

# Iterate over each unique texture and count occurrences in the 'image_name' column
for tex in texture['texture']:
    count = df['image_name'].str.contains(tex, regex=True).sum()  # Counts the number of True values
    count_values.append([tex, count])  # Store texture and its count

# Convert count_values to DataFrame for better display
count_df = pd.DataFrame(count_values, columns=['Texture', 'Count'])

In [ ]:
count_df

In [ ]:
for row in count_df.itertuples():
    if row.Count < 1000:
        print("Dropped " + row.Texture)
        texture.drop(texture[texture['texture'] == row.Texture].index, inplace=True)

texture

In [ ]:
import pandas as pd
import os
import re
from io import StringIO

In [ ]:
df = pd.read_csv("labels/Anno_coarse/list_bbox.txt", delim_whitespace=True, header = 1)
df.head()

In [ ]:
file_path = 'labels/Anno_coarse/list_attr_cloth.txt'

# Prepare to read the file and transform lines
transformed_lines = []
with open(file_path, 'r') as file:
    for line in file:
        # Strip leading/trailing whitespace and replace spaces before non-space characters
        transformed_line = re.sub(r' (?=\S)(?!\d)', '_', line.strip())
        transformed_line = transformed_line.title()
        transformed_lines.append(transformed_line)


# Convert the list of transformed lines into a DataFrame
data = '\n'.join(transformed_lines)
data = StringIO(data)
# Adjusting separator as necessary based on actual data
raw_attributes = pd.read_csv(data, sep='\s+', header=1)
column_names = ['attribute_name', 'attribute_type']
raw_attributes.columns = column_names
raw_attributes.head(10)


In [ ]:
count_values = []

# Iterate over each unique texture and count occurrences in the 'image_name' column
for row in raw_attributes.itertuples():
    attribute_name = getattr(row, 'attribute_name')
    attribute_type = getattr(row, 'attribute_type')
    count = df['image_name'].str.contains(attribute_name, regex=True).sum()  # Counts the number of True values
    count_values.append([attribute_name, attribute_type, count])  # Store texture and its count

# Convert count_values to DataFrame for better display
count_df = pd.DataFrame(count_values, columns=['attribute_name', 'attribute_type', 'count'])

In [ ]:
count_df

In [ ]:
selected_rows = []  # A list to hold the rows that meet the condition
for row in count_df.itertuples():
    if row.count > 1000:
        selected_rows.append(row)  # Append the row as a dictionary

attribute = pd.DataFrame(selected_rows)
attribute.drop(columns=['Index'], inplace=True)
attribute

In [ ]:
texture = attribute[attribute['attribute_type'] == 1]
fabric = attribute[attribute['attribute_type'] == 2]
shape = attribute[attribute['attribute_type'] == 3]
part = attribute[attribute['attribute_type'] == 4]
style = attribute[attribute['attribute_type'] == 5]    

In [ ]:
style